In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def parse_ltspice_line(line):
    split_line = line.split('\t')
    f = float(split_line[0])
    mag = float(split_line[1].split(',')[0].replace('dB', '').replace('(', ''))
    arg = float(split_line[1].split(',')[1].replace('°)', ''))
    return (f, mag, arg)

def parse_file(filename):
    f = open(filename, "r")
    f.readline()
    freq = []
    mag = []
    arg = []
    for line in f:
        f, m, a = parse_ltspice_line(line)
        freq.append(f)
        mag.append(m)
        arg.append(corregir_fase(a))
    return freq, mag, arg

def corregir_fase(fase):
    fase = fase%360
    if fase < 0:
        return 360 - fase
    else:
        return fase

#Levanto datos de la simulacion

ltspice_freq, ltspice_mag, ltspice_arg = parse_file('sim.txt')

#Levanto y proceso datos de las mediciones del osciloscopio

df_osc = pd.read_csv('mediciones.csv', usecols = ['frecuencia_hz','vin_rms', 'vout_rms', 'delay_us'])
osc_freq = df_osc['frecuencia_hz']
osc_mag = [20*np.log10(x[1]/x[0]) for x in zip(df_osc['vin_rms'], df_osc['vout_rms'])] #Calculo la ganancia y convierto a dB
osc_arg = [ corregir_fase(-360*x[0]*x[1]/10**6) for x in zip(df_osc['frecuencia_hz'], df_osc['delay_us'])] #Calculo la fase

#Levanto datos del analizador de audio (modulo)

df_an_mag = pd.read_csv('mediciones_analizador_modulo.csv', usecols = ['freq','mag'])
an_freq_mag = df_an_mag['freq']
an_mag = df_an_mag['mag']

#Levanto datos del analizador de audio (fase)

df_an_arg = pd.read_csv('mediciones_analizador_fase.csv', usecols = ['freq','arg'])
an_freq_arg = df_an_arg['freq']
an_arg = [corregir_fase(x) for x in df_an_arg['arg']]


#Grafico modulo

plt.xscale('log')
plt.plot(ltspice_freq, ltspice_mag, label='Simulacion')
plt.scatter(osc_freq, osc_mag, marker='x', color='r', label='Medicion con Osciloscopio')
plt.scatter(an_freq_mag, an_mag, marker='x', color='y', label='Medicion con Analizador')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Ganancia [dB]')
plt.legend()
plt.grid()
plt.show()

#Grafico modulo cerca de la banda de paso

plt.xlim(4000, 9000)
plt.ylim(-10, 0.5)
plt.xscale('log')
plt.plot(ltspice_freq, ltspice_mag, label='Simulacion')
plt.scatter(osc_freq, osc_mag, marker='x', color='r', label='Medicion con Osciloscopio')
plt.scatter(an_freq_mag, an_mag, marker='x', color='y', label='Medicion con Analizador')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Ganancia [dB]')
plt.legend()
plt.grid()
plt.show()

#Grafico error cerca de la banda de paso

err_mod_osc = [100*(x[0] - x[1])/x[1] for x in zip(osc_mag, ltspice_mag)]

plt.xlim(4000, 9000)
plt.ylim(-10, 0.5)
plt.xscale('log')
plt.plot(ltspice_freq, ltspice_mag, label='Simulacion')
plt.scatter(err_mod_osc, osc_mag, marker='x', color='r', label='Medicion con Osciloscopio')
plt.scatter(an_freq_mag, an_mag, marker='x', color='y', label='Medicion con Analizador')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Ganancia [dB]')
plt.legend()
plt.grid()
plt.show()


#Grafico fase
plt.ylim(60, 300)
plt.xscale('log')
plt.plot(ltspice_freq, ltspice_arg, label='Simulacion')
plt.scatter(osc_freq, osc_arg, marker='x', color='r', label='Medicion con Osciloscopio')
plt.scatter(an_freq_arg, an_arg, marker='x', color='y', label='Medicion con Analizador')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Fase [Grados]')
plt.yticks([90, 180, 270])
plt.legend()
plt.grid()
plt.show()

#Grafico fase cerca de la banda de paso

plt.xlim(3000, 10000)
plt.ylim(60, 300)
plt.xscale('log')
plt.plot(ltspice_freq, ltspice_arg, label='Simulacion')
plt.scatter(osc_freq, osc_arg, marker='x', color='r', label='Medicion con Osciloscopio')
plt.scatter(an_freq_arg, an_arg, marker='x', color='y', label='Medicion con Analizador')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Fase [Grados]')
plt.yticks([90, 180, 270])
plt.legend()
plt.grid()
plt.show()




SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (1979320328.py, line 84)